# Computation-Aware Gaussian Processes

In [9]:
# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import tueplots
from tueplots import bundles
plt.rcParams.update(bundles.neurips2022())

# Scientific computing
import numpy as np
import networkx as nx

## Learning to Navigate

### Landscape 

In [2]:
# Himmelblau's function
f1 = lambda x, y: (x**2 + y - 11) ** 2 + (x + y**2 - 7) ** 2
f = lambda x, y: f1(8 * (x -0.5), 8 * (y -0.5))


In [3]:
# Evaluate function for plotting
x_range = [0, 1]
y_range = [0, 1]

X = np.linspace(0, 1.0, 1000)
Y = np.linspace(0, 1.0, 1000)
X, Y = np.meshgrid(X, Y)
Z = f(X,Y)

### Planning

In [4]:
n_grid = 100

def coord_trafo(node):
    return (2.0 * node[0] /  n_grid, node[1] / n_grid * 2.0 / np.sqrt(3.0))

def inverse_coord_trafo(x):
    return (int(x[0] * 0.5 * n_grid), int(x[1] * np.sqrt(3) / 2.0 * n_grid))

# Graph
# G = nx.hexagonal_lattice_graph(n_grid, n_grid, with_positions=True)
G = nx.triangular_lattice_graph(n_grid, n_grid, with_positions=True)
G = G.to_directed()
nx.set_node_attributes(G, {n: coord_trafo(G.nodes[n]["pos"]) for n in G.nodes()}, "pos")


In [5]:
# Path finding
def dist(a, b):
    """Heuristic for A* algorithm."""
    (x1, y1) = G.nodes[a]["pos"]
    (x2, y2) = G.nodes[b]["pos"]
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def cost(a, b):
    """Cost moving from node a to b."""
    diff = f(b["pos"][0], b["pos"][1]) - f(a["pos"][0], a["pos"][1])
    if diff >= 0:
        return (diff + 1.0) ** 2
    else:
        return 1 / (-diff + 1.0) ** 2

nx.set_edge_attributes(
    G, {e: cost(G.nodes[e[0]], G.nodes[e[1]]) for e in G.edges()}, "cost"
)

start = inverse_coord_trafo((0.1, 0.1))
end = inverse_coord_trafo((0.9, 0.2))
path_nodes = nx.astar_path(G, start, end, heuristic=dist, weight="cost")
path = np.vstack([G.nodes[node]["pos"] for node in path_nodes])
# path = np.vstack(nx.dijkstra_path(G, start, end, weight="cost"))

length = nx.astar_path_length(G, start, end, heuristic=None, weight="cost")

In [8]:
with plt.rc_context(bundles.neurips2022(usetex=True, rel_width=1.0)):

    fig, ax = plt.subplots()
    ax.contourf(X, Y, Z, levels=50, cmap="terrain")
    ax.scatter(*G.nodes[start]["pos"], color="red")
    ax.scatter(*G.nodes[end]["pos"], color="red", marker="*")
    ax.plot(path[:, 0], path[:, 1], color="red")
    ax.set_aspect("equal")
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
# nx.draw_networkx_edges(G, pos={node: G.nodes[node]["pos"] for node in G.nodes}, ax=ax)
print("Path length: ", length)

Path length:  587.8708396306465


## Gaussian Processes

## Scaling to Large Datasets

## Computation-Aware Inference

### Design Your Own Policy